## Very Rough Methodology for Backtesting Different Models + Simulating Performance

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError


In [4]:
pd.set_option('max_colwidth', 100) 
pd.set_option('display.max_rows', 500)

In [5]:
full_df = pd.read_csv('/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/YQ_Enriched/Buys/2024_QTR2.csv', index_col='Unnamed: 0')

In [6]:
full_df_2 = pd.read_csv('/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/YQ_Enriched/Buys/2024_QTR1.csv', index_col='Unnamed: 0')

In [7]:
concatenated_df = pd.concat([full_df, full_df_2])


In [8]:
df_unique = concatenated_df.drop_duplicates(subset=['ticker', 'reporting_date', 'total_amount_bought_or_sold'])

In [9]:
df_unique = df_unique[(df_unique['dof_price'] > 0.5) & (df_unique['purchase_price'] > 0.5)]

In [10]:
df_unique['purchase_price_vs_price_on_dof'] = ((df_unique['purchase_price'] - df_unique['dof_price']) / df_unique['dof_price'])*100

In [11]:
df_unique.reset_index(inplace=True)

In [12]:
numerics = df_unique[['index', 'total_amount_bought_or_sold', 'total_dollar_bought_or_sold',
       'amount_owned_following', 'shares_percent_increased', 'dof_price',
       'vix_on_dof', 'tnx_on_dof', 'minus_10_price', '10_day_delta', 'purchase_price_vs_price_on_dof',
       '30_day_delta', 'minus_30_price', 'dof_vs_30_avg', 'dof_vs_30_actual']]

In [13]:
numerics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3058 entries, 0 to 3057
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           3058 non-null   int64  
 1   total_amount_bought_or_sold     3058 non-null   int64  
 2   total_dollar_bought_or_sold     3058 non-null   int64  
 3   amount_owned_following          3058 non-null   float64
 4   shares_percent_increased        3058 non-null   float64
 5   dof_price                       3058 non-null   float64
 6   vix_on_dof                      3058 non-null   float64
 7   tnx_on_dof                      3058 non-null   float64
 8   minus_10_price                  3058 non-null   float64
 9   10_day_delta                    3058 non-null   float64
 10  purchase_price_vs_price_on_dof  3058 non-null   float64
 11  30_day_delta                    3058 non-null   float64
 12  minus_30_price                  30

In [14]:
#multiply the percentages by 100 for readability
numerics['dof_vs_30_actual'] = numerics['dof_vs_30_actual'] *100
numerics['dof_vs_30_avg'] = numerics['dof_vs_30_avg'] *100

/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26051/2620116522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerics['dof_vs_30_actual'] = numerics['dof_vs_30_actual'] *100
/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26051/2620116522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerics['dof_vs_30_avg'] = numerics['dof_vs_30_avg'] *100


In [15]:
# split data into features and target variable
X = numerics[['total_amount_bought_or_sold', 'total_dollar_bought_or_sold',
       'amount_owned_following', 'shares_percent_increased', 'dof_price',
       'vix_on_dof', 'tnx_on_dof', 'minus_10_price', '10_day_delta', 'purchase_price_vs_price_on_dof',
       '30_day_delta', 'minus_30_price']]
y = numerics['dof_vs_30_actual']
y_alternate = numerics['dof_vs_30_avg']



In [16]:
len(X)

3058

In [17]:
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError

@tf.keras.utils.register_keras_serializable()
def custom_loss_1(y_true, y_pred):
    # cap y_true and y_pred at 200
    y_true_capped = tf.minimum(y_true, 200)
    y_pred_capped = tf.minimum(y_pred, 200)

    # calculate the base Mean Squared Error with capped values
    mse = MeanSquaredError()(y_true_capped, y_pred_capped)

    # calculate the penalty for values above 200
    penalty_factor_above_200 = 0.01
    penalty_above_200 = penalty_factor_above_200 * tf.maximum(y_true - 200, 0) * tf.maximum(y_pred - 200, 0)

    # penalty factor for false negatives (i.e., y_pred is negative while y_true is positive)
    false_negative_penalty_factor = 10.0

    # identify false negatives and apply penalty
    false_negatives = tf.logical_and(tf.greater(y_true, 0), tf.less(y_pred, 0))
    false_negative_penalty = tf.where(false_negatives, false_negative_penalty_factor, 1.0)

    # combine MSE with the penalties
    return mse + penalty_above_200 * false_negative_penalty



In [18]:
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
@tf.keras.utils.register_keras_serializable()
def custom_loss_2(y_true, y_pred):
    # high penalty for predicting positive when actual is negative
    high_penalty = 10

    # lower penalty for positive errors, regardless of the magnitude
    low_penalty = 0.5

    # determine conditions
    negative_actual_positive_pred = (y_true < 0) & (y_pred > 0)
    positive_error = (y_pred > y_true) & (y_true > 0)

    # standard Mean Squared Error
    mse = MeanSquaredError()(y_true, y_pred)

    # initialize a base penalty tensor with ones
    penalties = tf.ones_like(y_true)

    # apply high penalty for negative actual and positive prediction
    penalties = tf.where(negative_actual_positive_pred, penalties * high_penalty, penalties)

    # apply a lower penalty for positive errors
    penalties = tf.where(positive_error, penalties * low_penalty, penalties)

    # calculate final loss
    final_loss = mse * penalties

    return final_loss


In [19]:
 #custom loss function

@tf.keras.utils.register_keras_serializable()
def custom_loss_simple(y_true, y_pred):
    # penalty multiplier for crossing zero
    penalty = 5

     #check if true and predicted values are on opposite sides of zero
    sign_diff = tf.sign(y_true) != tf.sign(y_pred)

    # standard Mean Squared Error
    mse = tf.keras.losses.MSE(y_true, y_pred)

    # apply penalty for crossing zero
    return mse * (1 + penalty * tf.cast(sign_diff, tf.float32))


In [20]:
import tensorflow as tf
import numpy as np

def custom_quantile_loss(y_true, y_pred):
    # calculate the error
    err = y_true - y_pred
    
    # define the penalties
    penalty_very_positive = 0.05  # For errors > 20%
    penalty_high_positive = 0.1  # For errors between 10% and 20%
    penalty_moderate_positive = 0.2  # For errors between 5% and 10%
    penalty_low_positive = 0.4  # For errors between 0% and 5%
    penalty_negative = 1.0  # For negative errors (predicted > actual)
    
    # calculate the loss based on error ranges
    loss = tf.where(err >= 0.2, penalty_very_positive * err,
                    tf.where(err >= 0.1, penalty_high_positive * err,
                             tf.where(err >= 0.05, penalty_moderate_positive * err,
                                      tf.where(err >= 0, penalty_low_positive * err,
                                               penalty_negative * tf.abs(err)))))
    
    # return the mean loss
    return tf.reduce_mean(loss)



In [21]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable()
def custom_loss_neg_punishing(y_true, y_pred):
    # penalty multiplier for incorrect positive predictions
    penalty = 5

    # check if true values are negative and predicted values are positive
    incorrect_positive = (y_true < 0) & (y_pred > 0)

    # standard Mean Squared Error
    mse = tf.keras.losses.MSE(y_true, y_pred)

    # apply penalty for incorrect positive predictions
    return mse * (1 + penalty * tf.cast(incorrect_positive, tf.float32))


In [22]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable()
def quantile_loss(y_true, y_pred):
    """
    Calculate the quantile loss with a fixed quantile of 0.2.

    Parameters:
    y_true (Tensor): The true values.
    y_pred (Tensor): The predicted values.

    Returns:
    Tensor: The quantile loss.
    """
    q = 0.2
    e = y_true - y_pred
    loss = tf.reduce_mean(tf.maximum(q * e, (q - 1) * e))
    return loss


In [23]:
@tf.keras.utils.register_keras_serializable()
def custom_loss_piecewise(y_true, y_pred):
    # calculate the Mean Absolute Error
    mae = mean_absolute_error(y_true, y_pred)
    
    # conditions for discounting the loss
    discount_1 = tf.cast(tf.logical_and(y_pred < 0, y_true < 0), tf.float32) * 0.5
    discount_2 = tf.cast(tf.logical_and(y_pred < 0, tf.logical_and(y_true >= 0, y_true <= 5)), tf.float32) * 0.3
    discount_3 = tf.cast(tf.logical_and(y_pred > 10, y_true > 10), tf.float32) * 0.7
    penalty_1 = tf.cast(tf.logical_and(y_pred > 10, y_true < 0), tf.float32) * 1.5
    
    # default discount is 1 (no discount)
    discount = tf.ones_like(y_true) - discount_1 - discount_2 - discount_3 + penalty_1
    
    # apply the discount to the MAE
    discounted_mae = mae * discount
    
    return tf.reduce_mean(discounted_mae)

In [24]:
@tf.keras.utils.register_keras_serializable()
def custom_loss_piecewise_med(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    
    # conditions for discounting the loss
    discount_1 = tf.cast(tf.logical_and(y_pred < 0, y_true < 0), tf.float32) * 0.8 #if you guess negative and it is negative, all good
    discount_2 = tf.cast(tf.logical_and(y_pred < 0, tf.logical_and(y_true >= 0, y_true <= 5)), tf.float32) * 0.2 #if you guess negative and it's between 0 and positive 5, minor discount
    discount_3 = tf.cast(tf.logical_and(y_pred > 10, y_true > 10), tf.float32) * 0.8 # if you guess over 10 and it's over 10, don't sweat it, you're good
    penalty_1 = tf.cast(tf.logical_and(y_pred > 10, y_true < 0), tf.float32) * 2 # if you guess over 10 and it's negative- bad dog, very bad dog
    
    # default discount is 1 (no discount)
    discount = tf.ones_like(y_true) - discount_1 - discount_2 - discount_3 + penalty_1
    
    # apply the discount to the MAE
    discounted_mae = mae * discount
    
    return tf.reduce_mean(discounted_mae)

In [26]:
@tf.keras.utils.register_keras_serializable()
def custom_loss_piecewise_med_slightly_less(y_true, y_pred):
    # calculate the Mean Absolute Error
    mae = mean_absolute_error(y_true, y_pred)
    
    # conditions for discounting the loss
    discount_1 = tf.cast(tf.logical_and(y_pred < 0, y_true < 0), tf.float32) * 0.85 #if you guess negative and it is negative, all good
    discount_2 = tf.cast(tf.logical_and(y_pred < 0, tf.logical_and(y_true >= 0, y_true <= 5)), tf.float32) * 0.3 #if you guess negative and it's between 0 and positive 5, minor discount
    discount_3 = tf.cast(tf.logical_and(y_pred > 10, y_true > 10), tf.float32) * 0.85 # if you guess over 10 and it's over 10, don't sweat it, you're good
    penalty_1 = tf.cast(tf.logical_and(y_pred > 10, y_true < 0), tf.float32) * 1.75 # if you guess over 10 and it's negative- bad dog, very bad dog
    
    # default discount is 1 (no discount)
    discount = tf.ones_like(y_true) - discount_1 - discount_2 - discount_3 + penalty_1
    
    # apply the discount to the MAE
    discounted_mae = mae * discount
    
    return tf.reduce_mean(discounted_mae)

## First, attach the predictions for each model to a copy of the dataframe

In [30]:
import joblib  
import numpy as np
dataframes_list = []


models_and_scalers = [
     ('/Users/rileybitterli/6_25_24_simple__purchase.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler.pkl', custom_loss_piecewise),
    ('/Users/rileybitterli/6_25_24_piecewise_med_punishing.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler.pkl', custom_loss_piecewise_med),
    ('/Users/rileybitterli/6_25_24_piecewise_high_punishing.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler.pkl', custom_loss_piecewise_high),
    ('/Users/rileybitterli/6_25_24_piecewise_med_low_punishing.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler.pkl', custom_loss_piecewise_med_slightly_less),
    #('/Users/rileybitterli/6_25_24_piecewise_med_punishing__reg_and_drop.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler_optimized.pkl', custom_loss_piecewise_med)
    ('/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Training_Data_Scraper/6_25_24_piecewise_med_punishing_optimized.h5', '/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Scalers/6_25_scaler_optimized.pkl', custom_loss_piecewise_med)
]


# placeholder for results
dfs_with_predictions = {}

for i, (model_path, scaler_path, custom_loss_fn) in enumerate(models_and_scalers, start=1):
    # load the model with its specific custom loss function
    model = tf.keras.models.load_model(model_path, custom_objects={'custom_loss': custom_loss_fn})
    
    scaler = joblib.load(scaler_path)
    
    X_scaled = scaler.transform(X)
    
    # predict
    predictions = model.predict(X_scaled).flatten()  # Flatten in case predictions are multidimensional
    
    # create a copy of df_unique and add a new column with predictions
    df_temp = df_unique.copy()
    df_temp['predictions'] = predictions
    
    # add the modified DataFrame to the list
    dataframes_list.append(df_temp)


96/96 [==============================] - 0s 350us/step


In [31]:
dataframes_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3058 entries, 0 to 3057
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           3058 non-null   int64  
 1   ticker                          3009 non-null   object 
 2   reporting_date                  3058 non-null   object 
 3   url                             3058 non-null   object 
 4   total_amount_bought_or_sold     3058 non-null   int64  
 5   total_dollar_bought_or_sold     3058 non-null   int64  
 6   amount_owned_following          3058 non-null   float64
 7   purchase_price                  3058 non-null   float64
 8   shares_percent_increased        3058 non-null   float64
 9   dof_price                       3058 non-null   float64
 10  vix_on_dof                      3058 non-null   float64
 11  tnx_on_dof                      3058 non-null   float64
 12  minus_10_price                  30

In [32]:
runner_df = dataframes_list[0]

## Define function

In [33]:
from yahooquery import Ticker
from pandas.tseries.offsets import BDay
import pandas as pd
import concurrent.futures
import requests

# adapted fetch_stock_prices function for multiprocessing
def fetch_stock_prices(args):
    ticker, reporting_date = args
    reporting_date = pd.to_datetime(reporting_date) + BDay(0)
    
    try:
        # initialize Ticker object
        stock = Ticker(ticker)
        
        # calculate target dates
        today = pd.Timestamp('today') - BDay(1)
        thirty_bdays_after = reporting_date + BDay(22)
        
        # determine the target date for fetching the second price
        if today <= thirty_bdays_after:
            target_date_2 = today   # Last business day before today
        else:
            target_date_2 = thirty_bdays_after
        
        # initialize results dictionary
        results = {
            'ticker': ticker,
            'reporting_date': reporting_date,
            'price_on_reporting_day': None,
            'price_on_target_day': None
        }
        
        # fetch price on the reporting day
        try:
            results['price_on_reporting_day'] = stock.history(start=reporting_date, end=reporting_date + BDay(2))['close'].mean()
            print('got one')
        except Exception as e:
            print(f"Error fetching reporting day price for {ticker}: {e}")
        
        # fetch price on the target date
        try:
            results['price_on_target_day'] = stock.history(start=target_date_2- BDay(1), end=target_date_2)['close'].mean()
        except Exception as e:
            print(f"Error fetching target day price for {ticker}: {e}")
   
    except (requests.exceptions.RequestException, IndexError, KeyError, ValueError) as e:
        print(f"Error fetching prices for {ticker}: {e}")

    
    return results


In [34]:
runner_df['reporting_date'] = pd.to_datetime(runner_df['reporting_date'])

In [35]:
runner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3058 entries, 0 to 3057
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   index                           3058 non-null   int64         
 1   ticker                          3009 non-null   object        
 2   reporting_date                  3058 non-null   datetime64[ns]
 3   url                             3058 non-null   object        
 4   total_amount_bought_or_sold     3058 non-null   int64         
 5   total_dollar_bought_or_sold     3058 non-null   int64         
 6   amount_owned_following          3058 non-null   float64       
 7   purchase_price                  3058 non-null   float64       
 8   shares_percent_increased        3058 non-null   float64       
 9   dof_price                       3058 non-null   float64       
 10  vix_on_dof                      3058 non-null   float64       
 11  tnx_

## multi processing so this doesn't take my whole life

In [36]:
# assuming runner_df is prepared with 'ticker' and 'reporting_date'
tickers = runner_df['ticker'].tolist()
reporting_dates = runner_df['reporting_date'].dt.strftime('%Y-%m-%d').tolist()

# prepare tasks for multiprocessing
tasks = list(zip(tickers, reporting_dates))

# use ThreadPoolExecutor to fetch prices concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_stock_prices, task) for task in tasks]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

# convert the results to a DataFrame
stock_prices_df = pd.DataFrame(results)


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
Error fetching reporting day price for nan: 'float' object is not iterable
Error fetching target day price for nan: 'float' object is not iterable
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got o

Failed to obtain crumb.  Ability to retrieve data will be significantly limited.


got one
got one
got one
got one
got one
got one


Failed to obtain crumb.  Ability to retrieve data will be significantly limited.


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


...

got one


Failed to obtain crumb.  Ability to retrieve data will be significantly limited.


got one


Failed to obtain crumb.  Ability to retrieve data will be significantly limited.


got one


In [37]:
len(results)

3058

In [43]:
for i, df in enumerate(dataframes_list):
    # ensure 'reporting_date' is in the correct format
    df['reporting_date'] = pd.to_datetime(df['reporting_date'])
    # merge stock prices based on 'ticker' and 'reporting_date'
    dataframes_list[i] = pd.merge(df, stock_prices_df, on=['ticker', 'reporting_date'], how='left')


In [44]:
dataframes_list[0]

index ticker reporting_date  \
0           0   SAIA     2024-05-06   
1           1   BOLD     2024-04-04   
2           1   BOLD     2024-04-04   
3           1   BOLD     2024-04-04   
4           1   BOLD     2024-04-04   
...       ...    ...            ...   
124193   2444     EG     2024-02-13   
124194   2445   MOFG     2024-03-11   
124195   2446    DVN     2024-03-04   
124196   2447   SATS     2024-03-08   
124197   2448   SATS     2024-03-12   

                                                                                url  \
0       https://sec.gov/Archives/edgar/data/1666277/0001562180-24-003850-index.html   
1       https://sec.gov/Archives/edgar/data/1666306/0000950170-24-041841-index.html   
2       https://sec.gov/Archives/edgar/data/1666306/0000950170-24-041841-index.html   
3       https://sec.gov/Archives/edgar/data/1666306/0000950170-24-041841-index.html   
4       https://sec.gov/Archives/edgar/data/1666306/0000950170-24-041841-index.html   
...                                                                             ...   
124193  https://sec.gov/Archives/edgar/data/1485529/0001485529-24-000002-index.html   
124194  https://sec.gov/Archives/edgar/data/1412665/0001412665-24-000049-index.html   
124195  https://sec.gov/Archives/edgar/data/1413122/0001127602-24-008256-index.html   
124196  https://sec.gov/Archives/edgar/data/1415404/0001415889-24-007467-index.html   
124197  https://sec.gov/Archives/edgar/data/1415404/0001415889-24-007697-index.html   

        total_amount_bought_or_sold  total_dollar_bought_or_sold  \
0                               500                       203500   
1                            200000                      3200000   
2                            200000                      3200000   
3                            200000                      3200000   
4                            200000                      3200000   
...                             ...                          ...   
124193                          500                       178605   
124194                         4447                        99997   
124195                        15000                       666300   
124196                        90000                      1188600   
124197                       110000                      1482800   

        amount_owned_following  purchase_price  shares_percent_increased  \
0                      900.000        407.0000                  0.357143   
1                  1181766.000          1.0000                  0.144742   
2                  1181766.000          1.0000                  0.144742   
3                  1181766.000          1.0000                  0.144742   
4                  1181766.000          1.0000                  0.144742   
...                        ...             ...                       ...   
124193               16920.000        357.2100                  0.028703   
124194               38345.587         22.4866                  0.103920   
124195             1999055.000         41.5300                  0.007448   
124196             1883387.000         13.1400                  0.045607   
124197             1883387.000         13.4800                  0.055182   

         dof_price  ...  dof_vs_30_avg  dof_vs_30_actual  \
0       404.343333  ...      -0.006874          0.023314   
1        12.893333  ...      -0.137433         -0.265770   
2        12.893333  ...      -0.137433         -0.265770   
3        12.893333  ...      -0.137433         -0.265770   
4        12.893333  ...      -0.137433         -0.265770   
...            ...  ...            ...               ...   
124193  371.193339  ...      -0.001196          0.032059   
124194   22.603333  ...      -0.004340         -0.036573   
124195   44.966667  ...       0.063804          0.132320   
124196   14.756667  ...      -0.062087         -0.086289   
124197   14.943333  ...      -0.079985         -0.091457   

        purchase_price_vs_price_on_dof  pre

## See which model performed the best

In [45]:
outcomes = (((dataframes_list[0]['price_on_target_day'] - dataframes_list[0]['price_on_reporting_day']) / dataframes_list[0]['price_on_reporting_day']) * 100)

In [46]:
for df in dataframes_list:
    df['total_profit_or_loss'] = ((df['price_on_target_day'] - df['price_on_reporting_day']) / df['price_on_reporting_day']) * 100

In [47]:
dataframes_list[0][dataframes_list[0]['total_profit_or_loss'] > 10].drop_duplicates('ticker').sort_values('total_profit_or_loss', ascending = False)

index ticker reporting_date  \
86330     422   SKYE     2024-02-02   
83554     114   ZIVO     2024-01-02   
108777   1442   NOTV     2024-01-25   
47350     848   ASPI     2024-04-29   
105585   1143   SMCI     2024-02-05   
108142   1286   ABEO     2024-02-07   
95105     989   HRTG     2024-02-21   
3228       79   HROW     2024-04-19   
6221      329   TSSI     2024-04-05   
83874     193     NN     2024-03-20   
89068     688   SHWZ     2024-01-16   
86379     462   JSPR     2024-02-12   
108200   1298    VFF     2024-03-18   
87039     504   SANA     2024-02-12   
105511   1085   CRMD     2024-03-14   
120558   2197    EGY     2024-03-13   
112229   1622   SNDA     2024-02-05   
16960     584    GME     2024-04-09   
88962     670    DYN     2024-01-11   
94167     894   PAYS     2024-03-07   
38675     762   CALC     2024-04-02   
109027   1451   GATO     2024-02-27   
86023     415   RLMD     2024-01-31   
109349   1496   CLSD     2024-02-09   
3444      164    ORN     2024-05-02   
108274   1320   GLSI     2024-02-20   
93113     855    CDE     2024-03-19   
4713      222   CDZI     2024-04-19   
83237      84   SNCR     2024-01-04   
105535   1129    EVC     2024-03-18   
115948   1949   GENK     2024-03-13   
108196   1293    KLG     2024-03-01   
115257   1868   CRBP     2024-02-06   
5859      307   LOVE     2024-04-18   
3572      196   USAU     2024-04-17   
115255   1866   EYPT     2024-01-11   
87204     534   STAA     2024-03-14   
42952     829   REED     2024-05-09   
83269      90   QRHC     2024-03-15   
108699   1415   TGLS     2024-03-11   
112980   1715   VCSA     2024-01-26   
89921     739   KALV     2024-01-05   
105534   1098   MAIA     2024-03-18   
108336   1350   ALXO     2024-03-18   
6837      403   ELDN     2024-05-13   
95142    1009     IE     2024-03-01   
94836     943   VERA     2024-02-01   
84188     230   GTES     2024-02-20   
82013     940   XERS     2024-05-10   
7782      496    AES     2024-04-15   
95071     962    CTV     2024-03-01   
108143   1287   TISI     2024-03-14   
119375   2046   JANL     2024-02-13   
89384     706   RZLT     2024-02-21   
86332     424   CART     2024-02-22   
6241      351   STCN     2024-04-17   
88583     638   RCUS     2024-01-31   
83198      73   MRAI     2024-01-17   
6275      360   VSTS     2024-05-06   
84049     209    PBF     2024-01-17   
108202   1301   PRGO     2024-03-01   
3069       64   CRVS     2024-05-07   
113398   1832   BTSG     2024-03-15   
108768   1431   EVLV     2024-03-07   
83882     194   MDRX     2024-03-04   
120683   2232   SHYF     2024-02-28   
48485     910   IZEA     2024-04-17   
115528   1915   RAVE     2024-03-13   
113293   1775    OPK     2024-02-28   
105608   1152   CAUD     2024-01-17   
938        26   SHEN     2024-05-07   
82304      40   LCTX     2024-02-08   
7582      463   MRBK     2024-05-03   
109341   1487    GEO     2024-03-14   
86334     426   FNWB     2024-02-06   
111449   1581   STKS     2024-03-19   
94937     954   CGON     2024-01-31   
84210     238   BICX     2024-01-17   
48228     883   AENT     2024-04-25   
105531   1092    ADV     2024-03-06   
88540     622    WVE     2024-01-30   
108207   1306   STGW     2024-02-28   
83698     142    BKD     2024-02-23   
88958     666    JXN     2024-03-05   
108656   1393   AUID     2024-01-24   
120095   2109    DMA     2024-01-12   
109350   1498     WT     2024-02-14   
201        16   CFFI     2024-04-25   
6214      322   VUZI     2024-04-23   
6826      399    PTN     2024-04-11   
112393   1636    DXR     2024-03-06   
25556     660   LOGI     2024-05-03   
109344   1491   SPHR     2024-02-28   
57         10   CRMT     2024-04-15   
7708      479   XPEL     2024-05-09   
6913      439    EAF     2024-04-18   
1218       46   ENPH     2024-05-02   
8456      514   SMTC     2024-04-08   
82178       5     AN     2024-02-27   
95113     997   PTWO     2024-03-26   
7784      498   ATNI     2024-04-30   
83782     157    MDV     2024-

In [56]:
updated_dataframes_list = []
for df in dataframes_list:
    filtered_df = df[df['predictions'] > 4]
    deduped_df = filtered_df.drop_duplicates(subset=['ticker'], keep='first')
    updated_dataframes_list.append(deduped_df)

In [57]:
# initialize variables to track the best performing models for each metric
best_total_profit_or_loss = float('-inf')
best_total_profit_or_loss_model = None
best_number_of_trades = float('-inf')
best_number_of_trades_model = None
best_ROI = float('-inf')
best_ROI_model = None

# loop through each DataFrame and calculate the metrics
for i, df in enumerate(updated_dataframes_list):
    
    # calculate metrics
    total_profit_or_loss = df['total_profit_or_loss'].sum()
    number_of_trades = df.shape[0]
    ROI = total_profit_or_loss / (number_of_trades * 100) if number_of_trades > 0 else 0

    # print metrics for the current model
    print(f"Model {i+1}:")
    print(f"- Total Profit or Loss: {total_profit_or_loss}")
    print(f"- Number of Trades: {number_of_trades}")
    print(f"- ROI: {ROI}")
    print()

    # update best performing models
    if total_profit_or_loss > best_total_profit_or_loss:
        best_total_profit_or_loss = total_profit_or_loss
        best_total_profit_or_loss_model = i+1

    if number_of_trades > best_number_of_trades:
        best_number_of_trades = number_of_trades
        best_number_of_trades_model = i+1

    if ROI > best_ROI:
        best_ROI = ROI
        best_ROI_model = i+1

# print out which model performed best for each metric
print(f"Best Total Profit or Loss: Model {best_total_profit_or_loss_model} with {best_total_profit_or_loss}")
print(f"Best Number of Trades: Model {best_number_of_trades_model} with {best_number_of_trades} trades")
print(f"Best ROI: Model {best_ROI_model} with {best_ROI}")


Model 1:
- Total Profit or Loss: 463.0719674184231
- Number of Trades: 282
- ROI: 0.016420991752426353

Model 2:
- Total Profit or Loss: 617.8077919999657
- Number of Trades: 404
- ROI: 0.015292272079207072

Model 3:
- Total Profit or Loss: 479.0687864753962
- Number of Trades: 105
- ROI: 0.045625598711942494

Model 4:
- Total Profit or Loss: 303.1395660555488
- Number of Trades: 366
- ROI: 0.008282501804796416

Model 5:
- Total Profit or Loss: 7.077182741981377
- Number of Trades: 28
- ROI: 0.002527565264993349

Best Total Profit or Loss: Model 2 with 617.8077919999657
Best Number of Trades: Model 2 with 404 trades
Best ROI: Model 3 with 0.045625598711942494


In [186]:
best = updated_dataframes_list[1].sort_values('total_profit_or_loss', ascending= False)

In [187]:
best

index ticker reporting_date  \
2168    422   SKYE     2024-02-02   
2501    663   JSPR     2024-02-09   
4817   2286    EVC     2024-03-07   
241     165   CRVS     2024-05-07   
437     275   VABK     2024-04-29   
614     402   CFFI     2024-05-02   
971     635   PSTV     2024-05-10   
4848   2304    LSF     2024-03-18   
2836    871   NVVE     2024-02-08   
3277   1155   AXDX     2024-01-25   
3696   1465   IMRX     2024-03-19   
418     264   ABEO     2024-04-26   
1059    678     GL     2024-04-26   
1328    838   HNVR     2024-04-26   

                                                                              url  \
2168  https://sec.gov/Archives/edgar/data/1555952/0001209191-24-002643-index.html   
2501  https://sec.gov/Archives/edgar/data/1822834/0000921895-24-000337-index.html   
4817  https://sec.gov/Archives/edgar/data/1109116/0000950170-24-027689-index.html   
241   https://sec.gov/Archives/edgar/data/1626971/0001610717-24-000231-index.html   
437   https://sec.gov/Archives/edgar/data/1726848/0000950170-24-049570-index.html   
614    https://sec.gov/Archives/edgar/data/913341/0001415889-24-011996-index.html   
971   https://sec.gov/Archives/edgar/data/1406815/0000950170-24-057454-index.html   
4848  https://sec.gov/Archives/edgar/data/1650696/0001437749-24-008321-index.html   
2836  https://sec.gov/Archives/edgar/data/1836875/0001836875-24-000029-index.html   
3277  https://sec.gov/Archives/edgar/data/1194786/0001104659-24-006674-index.html   
3696  https://sec.gov/Archives/edgar/data/1870062/0001790340-24-000026-index.html   
418   https://sec.gov/Archives/edgar/data/1715291/0001493152-24-016062-index.html   
1059  https://sec.gov/Archives/edgar/data/1201565/0001062993-24-008838-index.html   
1328  https://sec.gov/Archives/edgar/data/1910427/0001619964-24-000026-index.html   

      total_amount_bought_or_sold  total_dollar_bought_or_sold  \
2168                      5206074                     12026030   
2501                       350000                      4532500   
4817                       100000                       167250   
241                        577634                       999999   
437                          1000                        24986   
614                          1000                        39465   
971                         12255                        25000   
4848                        50000                       139468   
2836                        25000                        50000   
3277                        13333                        13333   
3696                        20000                        55326   
418                          5608                        17609   
1059                         2000                       156940   
1328                          200                         3356   

      amount_owned_following  purchase_price  shares_percent_increased  \
2168            7.943520e+06          2.3100                  0.395911   
2501            1.188500e+06         12.9500                  0.227494   
4817            1.402170e+06          1.6725                  0.066570   
241             1.490119e+06          1.7312                  0.279353   
437             1.177900e+04         24.9900                  0.078253   
614             5.635000e+03         39.4400                  0.150716   
971             3.642100e+04          2.0400                  0.251767   
4848            1.790570e+05          2.7140                  0.218286   
2836            9.060784e+04          2.0000                  0.216248   
3277            2.092700e+04          1.0000                  0.389171   
3696            2.281852e+06          2.7663                  0.008689   
418             2.338680e+05          3.1400                  0.023418   
1059            7.030244e+03         78.4702                  0.221478   
1328            2.117100e+04         16.7800                  0.009358   

      dof_price  ...  10_day_delta  30_day_delta  minus_30_price  \
2168

In [ ]:
import smtplib
from email.message import EmailMessage
def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    # Establish a connection to Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login([insert your email address], '[insert your password]')
    server.send_message(msg)
    server.quit()

send_email('step 4', 'done')